**Fully cleaning the dataset**

**Importing data**


In [1]:
# imports
from IPython.display import clear_output
from PIL import Image
import os
import json
# clear
clear_output(wait=True)
# installing kaggle
!pip install -q kaggle
# user and key info
info = {"username":"jackbellamy","key":"3442898ba5e904ac3a48eb5b59398805"}
# opening file
json_object = json.dumps(info, indent=4)
with open("kaggle.json", "w") as outfile:
    outfile.write(json_object)
!mkdir ~/.kaggle
!cp kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json
# copying data to content file of kaggle which can be found on the left hand side of the screen
if not (os.path.exists("/content/nfl-big-data-bowl-2022")):
  !kaggle competitions download -c nfl-big-data-bowl-2022
  !mkdir nfl-big-data-bowl-2022
  !unzip nfl-big-data-bowl-2022.zip -d nfl-big-data-bowl-2022

 99% 771M/783M [00:12<00:00, 61.6MB/s]
100% 783M/783M [00:12<00:00, 65.9MB/s]
Archive:  nfl-big-data-bowl-2022.zip
  inflating: nfl-big-data-bowl-2022/PFFScoutingData.csv  
  inflating: nfl-big-data-bowl-2022/games.csv  
  inflating: nfl-big-data-bowl-2022/players.csv  
  inflating: nfl-big-data-bowl-2022/plays.csv  
  inflating: nfl-big-data-bowl-2022/tracking2018.csv  
  inflating: nfl-big-data-bowl-2022/tracking2019.csv  
  inflating: nfl-big-data-bowl-2022/tracking2020.csv  


**Serialisation**

In [2]:
import pandas as pd
# reading the data to variables
scouting = pd.read_csv("/content/nfl-big-data-bowl-2022/PFFScoutingData.csv")
games = pd.read_csv("/content/nfl-big-data-bowl-2022/games.csv")
players = pd.read_csv("/content/nfl-big-data-bowl-2022/players.csv")
plays = pd.read_csv("/content/nfl-big-data-bowl-2022/plays.csv")
tracking2018 = pd.read_csv("/content/nfl-big-data-bowl-2022/tracking2018.csv")
tracking2019 = pd.read_csv("/content/nfl-big-data-bowl-2022/tracking2019.csv")
tracking2020 = pd.read_csv("/content/nfl-big-data-bowl-2022/tracking2020.csv")

**Generalised cleaning functions**

**Abbreviations to full names**

In [3]:
def abbrvs(csv,column,abbrv,fullname):
  # new names
  type_name = []
  # looping through
  for i in csv[column]:
    # filtering out NaN values
    if i in abbrv:
      index = abbrv.index(i)
      type_name.append(fullname[index])
    else:
      type_name.append(i)
  return type_name

**Team and number splitting**

In [4]:
def team_number_splitting(csv,column,symbol):
  # splitting to team and number
  split_team = []
  split_numbers = []
  # splitting the data
  split = csv[column].str.split(symbol, expand = True)
  # loop for each
  for i in range(0,len(csv)):
    # specific data
    split.iloc[i]
    # number of NaN
    nan_num = split.iloc[i].isna().sum()
    max = len(split.iloc[i])
    if max-nan_num == 0:
      split_team.append(csv[column][i])
      split_numbers.append(csv[column][i])
    else:
      info = []
      for j in range(0,(max-nan_num)):
        split_i = split[j][i].split(" ")
        info.append(split_i)
      # team name
      split_team.append(info[0][0])
      # numbers
      nums = []
      for k in range(0,max-nan_num):
        num = info[k][1]
        nums.append(num)
      split_numbers.append(nums)
  return split_team,split_numbers

**Number splitting**

In [5]:
def number_splitting(csv,column,symbol,n):
  split = csv[column].str.split(symbol, n = n, expand = True)
  return split

**Pandas to number**

In [6]:

def pd_to_num(csv,split,column_names):
  split.columns = column_names
  # adding columns
  for name in column_names:
    csv[name] = pd.to_numeric(split[name])
  return

**Scouting** 


In [7]:
# before
scouting.head()

,gameId,playId,snapDetail,snapTime,operationTime,hangTime,kickType,kickDirectionIntended,kickDirectionActual,returnDirectionIntended,returnDirectionActual,missedTackler,assistTackler,tackler,kickoffReturnFormation,gunners,puntRushers,specialTeamsSafeties,vises,kickContactType
0,2018090600,37,NaN,NaN,NaN,3.85,D,R,R,NaN,NaN,NaN,NaN,NaN,8-0-2,NaN,NaN,PHI 23; PHI 27,NaN,NaN
1,2018090600,366,OK,0.84,2.12,4.46,N,C,C,C,R,PHI 57,NaN,PHI 54,NaN,PHI 18; PHI 29,NaN,NaN,ATL 83; ATL 27; ATL 34; ATL 21,CC
2,2018090600,658,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,PHI 58,NaN,NaN
3,2018090600,677,NaN,NaN,NaN,4.06,D,R,R,C,C,ATL 83,ATL 22,ATL 27,8-0-2,NaN,NaN,ATL 17; ATL 22,NaN,NaN
4,2018090600,872,OK,0.84,2.00,4.35,N,C,L,NaN,NaN,NaN,NaN,NaN,NaN,PHI 18; PHI 29,ATL 85,ATL 37,ATL 83; ATL 34; ATL 21,BF


In [8]:
# abbreviations: 'kickType', 'kickDirectionIntended', 'kickDirectionActual', 'returnDirectIntended', 'returnDirectionActual', 'KickTypeContact'
# team number splitting: 'missedTackler', 'specialTeamsSafeties', 'Vises'
# number splitting: 'kickoffReturnFormation'

In [9]:
# abbreviations
# 'kickType'
csv = scouting
column = "kickType"
kick_types = ['D','F','K','O','P','Q','S','B','N','R','A']
kick_type_names = ['Deep','Flat','Free','Obvious','Pooch','Squib','Surprise','Deep','Normal','Rugby','Aussie']
# calling function
kick_type_fullnames = abbrvs(csv,column,kick_types,kick_type_names)
# adding column
scouting["kick_type_name"] = kick_type_fullnames

# 'kickDirectionIntended', 'kickDirectionActual', 'returnDirectIntended', 'returnDirectionActual', 'KickTypeContact'

In [10]:
# team and numbers
# 'missedTackler'
csv = scouting

column = "missedTackler"
symbol = "; "
# calling function
missed_tackler_team, missed_tackler_numbers = team_number_splitting(csv,column,symbol)
# adding
scouting["missed_tackler_team"] = missed_tackler_team
scouting["missed_tackler_numbers"] = missed_tackler_numbers

# 'specialTeamsSafeties'
column = "specialTeamsSafeties"
symbol = "; "
# calling function
specialTeamsSafeties_team, specialTeamsSafeties_numbers = team_number_splitting(csv,column,symbol)
# adding
scouting["special_teams_safeties_team"] = specialTeamsSafeties_team
scouting["special_teams_safeties_numbers"] = specialTeamsSafeties_numbers

# 'Vises'
column = "vises"
symbol = "; "
# calling function
vises_team, vises_numbers = team_number_splitting(csv,column,symbol)
# adding
scouting["vises_team"] = vises_team
scouting["vises_numbers"] = vises_numbers

In [11]:
# number splitting
# 'kickoffReturnFormation'
csv = scouting
column = "kickoffReturnFormation"
symbol = "-"
n = 2
column_names = ['formation_front','formation_mid','formation_back']
# calling function
formation_split = number_splitting(csv,column,symbol,n)
# calling function
pd_to_num(csv,formation_split,column_names)

In [12]:
# after
scouting.head()

,gameId,playId,snapDetail,snapTime,operationTime,hangTime,kickType,kickDirectionIntended,kickDirectionActual,returnDirectionIntended,...,kick_type_name,missed_tackler_team,missed_tackler_numbers,special_teams_safeties_team,special_teams_safeties_numbers,vises_team,vises_numbers,formation_front,formation_mid,formation_back
0,2018090600,37,NaN,NaN,NaN,3.85,D,R,R,NaN,...,Deep,NaN,NaN,PHI,"[23, 27]",NaN,NaN,8.0,0.0,2.0
1,2018090600,366,OK,0.84,2.12,4.46,N,C,C,C,...,Normal,PHI,[57],NaN,NaN,ATL,"[83, 27, 34, 21]",NaN,NaN,NaN
2,2018090600,658,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,PHI,[58],NaN,NaN,NaN,NaN,NaN
3,2018090600,677,NaN,NaN,NaN,4.06,D,R,R,C,...,Deep,ATL,[83],ATL,"[17, 22]",NaN,NaN,8.0,0.0,2.0
4,2018090600,872,OK,0.84,2.00,4.35,N,C,L,NaN,...,Normal,NaN,NaN,ATL,[37],ATL,"[83, 34, 21]",NaN,NaN,NaN


**Games**

In [13]:
# before
games.head()

,gameId,season,week,gameDate,gameTimeEastern,homeTeamAbbr,visitorTeamAbbr
0,2018090600,2018,1,09/06/2018,20:20:00,PHI,ATL
1,2018090900,2018,1,09/09/2018,13:00:00,BAL,BUF
2,2018090901,2018,1,09/09/2018,13:00:00,CLE,PIT
3,2018090902,2018,1,09/09/2018,13:00:00,IND,CIN
4,2018090903,2018,1,09/09/2018,13:00:00,MIA,TEN


In [14]:
# abbreviations: 'homeTeamAbbr' , 'visitorTeamAbbr'
# team number splitting:
# number splitting: 'gameDate', 'gameTimeEastern'

In [15]:
# abbreviations
# 'homeTeamAbbr', 'visitorTeamAbbr'

In [16]:
# number splitting
# 'gameDate'
csv = games

column = "gameDate"
symbol = "/"
n = 2
column_names = ['game_date_day','game_date_month','game_date_year']
# calling function
game_date_split = number_splitting(csv,column,symbol,n)
# calling function
pd_to_num(csv,game_date_split,column_names)

# 'gameTimeEastern'
column = "gameTimeEastern"
symbol = ":"
n = 2
column_names = ['game_time_hour','game_time_minute','game_time_second']
# calling function
game_time_split = number_splitting(csv,column,symbol,n)
# calling function
pd_to_num(csv,game_time_split,column_names)

In [17]:
# after
games.head()

,gameId,season,week,gameDate,gameTimeEastern,homeTeamAbbr,visitorTeamAbbr,game_date_day,game_date_month,game_date_year,game_time_hour,game_time_minute,game_time_second
0,2018090600,2018,1,09/06/2018,20:20:00,PHI,ATL,9,6,2018,20,20,0
1,2018090900,2018,1,09/09/2018,13:00:00,BAL,BUF,9,9,2018,13,0,0
2,2018090901,2018,1,09/09/2018,13:00:00,CLE,PIT,9,9,2018,13,0,0
3,2018090902,2018,1,09/09/2018,13:00:00,IND,CIN,9,9,2018,13,0,0
4,2018090903,2018,1,09/09/2018,13:00:00,MIA,TEN,9,9,2018,13,0,0


**Players**

In [18]:
# before
players.head()

,nflId,height,weight,birthDate,collegeName,Position,displayName
0,42901,6-1,208,1992-07-25,James Madison,SS,Dean Marlowe
1,43501,6-0,220,1994-08-11,Central Michigan,FS,Kavon Frazier
2,43534,6-0,205,1993-06-02,Illinois,SS,Clayton Fejedelem
3,43535,6-1,235,1992-12-22,Temple,MLB,Tyler Matakevich
4,44174,6-1,236,1993-02-22,Texas Tech,ILB,Sam Eguavoen


In [19]:
# abbreviations:
# team number splitting:
# number splitting: birth
# unique one is height

In [20]:
# 'birthDate'
# splitting the numbers
players['birth_year'] = pd.DatetimeIndex(players['birthDate']).year

In [21]:
# 'height'
height_split = players["height"].str.split("-", n = 1, expand = True)
# new column names
height_split.columns = ['Feet', 'Inch']
# adding new columns
players["height_feet"] = pd.to_numeric(height_split["Feet"])
players["height_inch"] = pd.to_numeric(height_split["Inch"])
# still have 73 feet
# getting rid of NaN values
players["height_inch"] = players["height_inch"].fillna(0)
# replacing heights that did not have '-' at start
for i in players["height_feet"]:
    if i > 7:
        j = 0.0833333 * i
        players["height_feet"].replace({i : j}, inplace=True)        
# inches to numerical value
players["height_inch"] = 0.0833333 * players["height_inch"]
# adding feet and inches to get one value
height_numerical = players["height_feet"] + players["height_inch"]
# adding new numerical value column
players["height_numerical"] = pd.to_numeric(height_numerical)

In [22]:
# after
players.head()

,nflId,height,weight,birthDate,collegeName,Position,displayName,birth_year,height_feet,height_inch,height_numerical
0,42901,6-1,208,1992-07-25,James Madison,SS,Dean Marlowe,1992.0,6.0,0.083333,6.083333
1,43501,6-0,220,1994-08-11,Central Michigan,FS,Kavon Frazier,1994.0,6.0,0.000000,6.000000
2,43534,6-0,205,1993-06-02,Illinois,SS,Clayton Fejedelem,1993.0,6.0,0.000000,6.000000
3,43535,6-1,235,1992-12-22,Temple,MLB,Tyler Matakevich,1992.0,6.0,0.083333,6.083333
4,44174,6-1,236,1993-02-22,Texas Tech,ILB,Sam Eguavoen,1993.0,6.0,0.083333,6.083333


**Plays**

In [23]:
# before
plays.head()

,gameId,playId,playDescription,quarter,down,yardsToGo,possessionTeam,specialTeamsPlayType,specialTeamsResult,kickerId,...,penaltyCodes,penaltyJerseyNumbers,penaltyYards,preSnapHomeScore,preSnapVisitorScore,passResult,kickLength,kickReturnYardage,playResult,absoluteYardlineNumber
0,2018090600,37,J.Elliott kicks 65 yards from PHI 35 to end zo...,1,0,0,PHI,Kickoff,Touchback,44966.0,...,NaN,NaN,NaN,0,0,NaN,66.0,NaN,40,45
1,2018090600,366,"(9:20) C.Johnston punts 56 yards to ATL 36, Ce...",1,4,4,PHI,Punt,Return,45603.0,...,UNSd,PHI 18,-15.0,0,0,NaN,56.0,5.0,36,18
2,2018090600,658,"(5:03) M.Bryant 21 yard field goal is GOOD, Ce...",1,4,3,ATL,Field Goal,Kick Attempt Good,27091.0,...,NaN,NaN,NaN,0,0,NaN,21.0,NaN,0,13
3,2018090600,677,M.Bosher kicks 64 yards from ATL 35 to PHI 1. ...,1,0,0,ATL,Kickoff,Return,37267.0,...,NaN,NaN,NaN,0,3,NaN,64.0,30.0,34,75
4,2018090600,872,"(:33) C.Johnston punts 65 yards to end zone, C...",1,4,18,PHI,Punt,Touchback,45603.0,...,NaN,NaN,NaN,0,3,NaN,65.0,NaN,45,45


In [24]:
# abbreviations: 'possessionTeam'
# team number splitting: 'penaltyJerseyNumbers'
# number splitting: 'gameClock'

In [25]:
# abbreviations
# 'possessionTeam'

In [26]:
# number splitting
# 'gameClock'
csv = plays

column = "gameClock"
symbol = ":"
n = 2
column_names = ['game_clock_minute','game_clock_second','game_clock_milli']
# calling function
game_time_split = number_splitting(csv,column,symbol,n)
# calling function
pd_to_num(csv,game_time_split,column_names)

In [27]:
# after
plays.head()

,gameId,playId,playDescription,quarter,down,yardsToGo,possessionTeam,specialTeamsPlayType,specialTeamsResult,kickerId,...,preSnapHomeScore,preSnapVisitorScore,passResult,kickLength,kickReturnYardage,playResult,absoluteYardlineNumber,game_clock_minute,game_clock_second,game_clock_milli
0,2018090600,37,J.Elliott kicks 65 yards from PHI 35 to end zo...,1,0,0,PHI,Kickoff,Touchback,44966.0,...,0,0,NaN,66.0,NaN,40,45,15,0,0
1,2018090600,366,"(9:20) C.Johnston punts 56 yards to ATL 36, Ce...",1,4,4,PHI,Punt,Return,45603.0,...,0,0,NaN,56.0,5.0,36,18,9,20,0
2,2018090600,658,"(5:03) M.Bryant 21 yard field goal is GOOD, Ce...",1,4,3,ATL,Field Goal,Kick Attempt Good,27091.0,...,0,0,NaN,21.0,NaN,0,13,5,3,0
3,2018090600,677,M.Bosher kicks 64 yards from ATL 35 to PHI 1. ...,1,0,0,ATL,Kickoff,Return,37267.0,...,0,3,NaN,64.0,30.0,34,75,5,1,0
4,2018090600,872,"(:33) C.Johnston punts 65 yards to end zone, C...",1,4,18,PHI,Punt,Touchback,45603.0,...,0,3,NaN,65.0,NaN,45,45,0,33,0


**Tracking**

In [28]:
# before
tracking2018.head()

,time,x,y,s,a,dis,o,dir,event,nflId,displayName,jerseyNumber,position,team,frameId,gameId,playId,playDirection
0,2018-12-30T21:25:32.200,41.32,29.45,4.36,1.33,0.43,130.42,128.44,None,39470.0,Justin Tucker,9.0,K,home,1,2018123000,36,right
1,2018-12-30T21:25:32.300,41.68,29.17,4.59,1.24,0.45,128.59,127.81,None,39470.0,Justin Tucker,9.0,K,home,2,2018123000,36,right
2,2018-12-30T21:25:32.400,42.05,28.88,4.74,0.99,0.47,124.47,128.15,None,39470.0,Justin Tucker,9.0,K,home,3,2018123000,36,right
3,2018-12-30T21:25:32.500,42.43,28.59,4.87,0.71,0.48,126.02,127.35,None,39470.0,Justin Tucker,9.0,K,home,4,2018123000,36,right
4,2018-12-30T21:25:32.600,42.84,28.31,4.96,0.79,0.50,131.71,124.75,None,39470.0,Justin Tucker,9.0,K,home,5,2018123000,36,right


In [29]:
# it is clean

**Overall, clean now except 'penaltyJerseyNumbers' and some abbrvs**

In [40]:
import pandas as pd
data_plays = plays
data_tracking2018 = tracking2018
data_tracking2019 = tracking2019
data_tracking2020 = tracking2020
import matplotlib.pyplot as plt
import matplotlib.colors as colors

#sack locations in order
#x: 47.09 R, 76.64 L
#y: 45.52 R, 26.56 L
#strip sack locations in order
#x: 66.8 R
#y: 22.85 R

#%% function def
import numpy as np

def return_vals(data, condition_name, comparison, condition_answer, return_collumn):
    conditions = ""
    for i in range(0, len(condition_name)-1):
        cn = condition_name[i]
        c = comparison[i]
        ca = condition_answer[i]
        if type(ca) is str:
            conditions += cn+" "+c+" "+"\""+ca+"\""+" "+"&"+" "
        else:
            ca = str(ca)
            conditions += cn+" "+c+" "+ca+" "+"&"+" "
    cn = condition_name[-1]
    c = comparison[-1]
    ca = condition_answer[-1]
    conditions += cn+" "+c+" "+"\""+ca+"\""
    conditions = str(conditions)
    return data.query(conditions)[return_collumn]

def to_list(data):
    data2 = []
    for i in data:
        data2.append(i.tolist())
    return data2

def truncate_colormap(cmap, minval=0.0, maxval=1.0, n=100):
    new_cmap = colors.LinearSegmentedColormap.from_list(
        'trunc({n},{a:.2f},{b:.2f})'.format(n=cmap.name, a=minval, b=maxval),
        cmap(np.linspace(minval, maxval, n)))
    return new_cmap

#%% return game and play ID
def s1(return_col1, return_col2):
    cn2 = ["specialTeamsPlayType","specialTeamsResult"]
    c2 = ["!=", "=="]
    ca2 = ["Extra Point","Non-Special Teams Result"]
    gameId = np.array(return_vals(data_plays, cn2, c2, ca2, return_col1))
    playId = np.array(return_vals(data_plays, cn2, c2, ca2, return_col2))
    return gameId, playId

gameId_all, playId_all = s1("gameId", "playId")

#%% get locations of specific events(can include other events)
def event_positions(event, direction):
    xpos_pass = []
    ypos_pass = []
    for i in range(0, len(playId_all)):
        #print(event,i)
        playId = playId_all[i]
        gameId = gameId_all[i]
        cn1 = ["gameId", "playId","team", "event", "playDirection"]
        c1 = ["==", "==", "==", "==", "=="]
        ca1 = [gameId, playId, "football", event, direction]
        if str(gameId)[:4] == "2018" :
            xpos = return_vals(data_tracking2018, cn1, c1, ca1, "x")
            ypos = return_vals(data_tracking2018, cn1, c1, ca1, "y")
        elif str(gameId)[:4] == "2019" :
            xpos = return_vals(data_tracking2019, cn1, c1, ca1, "x")
            ypos = return_vals(data_tracking2019, cn1, c1, ca1, "y")   
        else:
            xpos = return_vals(data_tracking2020, cn1, c1, ca1, "x")
            ypos = return_vals(data_tracking2020, cn1, c1, ca1, "y")  
        xpos_pass.append(xpos)
        ypos_pass.append(ypos)
    xpos_pass = to_list(xpos_pass)
    ypos_pass = to_list(ypos_pass)
    return xpos_pass, ypos_pass

#%%
xpos_pass_forward_l, ypos_pass_forward_l = event_positions("pass_forward", "left")
xpos_pass_shovel_l, ypos_pass_shovel_l = event_positions("pass_shovel", "left")
xpos_pass_incomplete_l, ypos_pass_incomplete_l = event_positions("pass_outcome_incomplete", "left")
xpos_pass_caught_l, ypos_pass_caught_l = event_positions("pass_outcome_caught", "left")
xpos_pass_td_l, ypos_pass_td_l = event_positions("pass_outcome_touchdown", "left")
xpos_pass_intercept_l, ypos_pass_intercept_l = event_positions("pass_outcome_interception", "left")


xpos_pass_forward_r, ypos_pass_forward_r = event_positions("pass_forward", "right")
xpos_pass_shovel_r, ypos_pass_shovel_r = event_positions("pass_shovel", "right")
xpos_pass_incomplete_r, ypos_pass_incomplete_r = event_positions("pass_outcome_incomplete", "right")
xpos_pass_caught_r, ypos_pass_caught_r = event_positions("pass_outcome_caught", "right")
xpos_pass_td_r, ypos_pass_td_r = event_positions("pass_outcome_touchdown", "right")
xpos_pass_intercept_r, ypos_pass_intercept_r = event_positions("pass_outcome_interception", "right")

#%% get locations of all plays
def all_locations():
    xpos = []
    ypos = []
    for i in range(0, len(playId_all)):
        #print("datapoint",i)
        playId = playId_all[i]
        gameId = gameId_all[i]
        cn = ["playId", "gameId", "team"]
        c = ["==", "==", "=="]
        ca = [playId, gameId, "football"]
        if str(gameId)[:4] == "2018" :
            xpos.append(return_vals(data_tracking2018, cn, c, ca, "x"))
            ypos.append(return_vals(data_tracking2018, cn, c, ca, "y"))
        elif str(gameId)[:4] == "2019" :
            xpos.append(return_vals(data_tracking2019, cn, c, ca, "x"))
            ypos.append(return_vals(data_tracking2019, cn, c, ca, "y"))
        else:
            xpos.append(return_vals(data_tracking2020, cn, c, ca, "x"))
            ypos.append(return_vals(data_tracking2020, cn, c, ca, "y"))
    xpos = to_list(xpos)
    ypos = to_list(ypos)
    return xpos, ypos
xpos, ypos = all_locations()

#%%pass forward caught
import matplotlib.pyplot as plt
from statistics import median
 
def return_path(event1_x, event1_y, event2_x, event2_y):
    outcomes_x = []
    outcomes_y = []
    for i in range(0, len(xpos)):
        condition1 = False
        condition2 = False
        xpositions = xpos[i]
        ypositions = ypos[i]
        if len(event1_x[i]) == 1: 
            condition1 = True
            xpos1 = (event1_x[i])[0]
            ypos1 = (event1_y[i])[0]
        if len(event2_x[i]) == 1:
            condition2 = True
            xpos2 = (event2_x[i])[0]
            ypos2 = (event2_y[i])[0]
        if condition1 == True & condition2 == True:
                indices_x1 = [i for i, x in enumerate(xpositions) if x == xpos1]
                indices_y1 = [i for i, y in enumerate(ypositions) if y == ypos1]
                for idx_a in indices_x1:
                    if idx_a in indices_y1:
                        idx_x1 = idx_a
                for idx_b in indices_y1:
                    if idx_b in indices_x1:
                        idx_y1 = idx_b
                indices_x2 = [i for i, x in enumerate(xpositions) if x == xpos2]
                indices_y2 = [i for i, y in enumerate(ypositions) if y == ypos2]
                for idx_c in indices_x2:
                    if idx_c in indices_y2:
                        idx_x2 = idx_c
                for idx_d in indices_y2:
                    if idx_d in indices_x2:
                        idx_y2 = idx_d
                xs = xpositions[idx_x1:idx_x2+1]
                ys = ypositions[idx_y1:idx_y2+1]
                outcomes_x.append(xs)
                outcomes_y.append(ys)
        else:
            outcomes_x.append("empty")
            outcomes_y.append("empty") 
    return outcomes_x, outcomes_y

#%%
xpf_c_l, ypf_c_l = return_path(xpos_pass_forward_l, ypos_pass_forward_l, xpos_pass_caught_l, ypos_pass_caught_l)
xpf_c_r, ypf_c_r = return_path(xpos_pass_forward_r, ypos_pass_forward_r, xpos_pass_caught_r, ypos_pass_caught_r)

xpf_int_l, ypf_int_l = return_path(xpos_pass_forward_l, ypos_pass_forward_l, xpos_pass_intercept_l, ypos_pass_intercept_l)
xpf_int_r, ypf_int_r = return_path(xpos_pass_forward_r, ypos_pass_forward_r, xpos_pass_intercept_r, ypos_pass_intercept_r)

xpf_inc_l, ypf_inc_l = return_path(xpos_pass_forward_l, ypos_pass_forward_l, xpos_pass_incomplete_l, ypos_pass_incomplete_l)
xpf_inc_r, ypf_inc_r = return_path(xpos_pass_forward_r, ypos_pass_forward_r, xpos_pass_incomplete_r, ypos_pass_incomplete_r)

xpf_td_l, ypf_td_l = return_path(xpos_pass_forward_l, ypos_pass_forward_l, xpos_pass_td_l, ypos_pass_td_l)
xpf_td_r, ypf_td_r = return_path(xpos_pass_forward_r, ypos_pass_forward_r, xpos_pass_td_r, ypos_pass_td_r)

xps_c_l, yps_c_l = return_path(xpos_pass_shovel_l, ypos_pass_shovel_l, xpos_pass_caught_l, ypos_pass_caught_l)
xps_c_r, yps_c_r = return_path(xpos_pass_shovel_r, ypos_pass_shovel_r, xpos_pass_caught_r, ypos_pass_caught_r)

xps_int_l, yps_int_l = return_path(xpos_pass_shovel_l, ypos_pass_shovel_l, xpos_pass_intercept_l, ypos_pass_intercept_l)
xps_int_r, yps_int_r = return_path(xpos_pass_shovel_r, ypos_pass_shovel_r, xpos_pass_intercept_r, ypos_pass_intercept_r)

xps_inc_l, yps_inc_l = return_path(xpos_pass_shovel_l, ypos_pass_shovel_l, xpos_pass_incomplete_l, ypos_pass_incomplete_l)
xps_inc_r, yps_inc_r = return_path(xpos_pass_shovel_r, ypos_pass_shovel_r, xpos_pass_incomplete_r, ypos_pass_incomplete_r)

xps_td_l, yps_td_l = return_path(xpos_pass_shovel_l, ypos_pass_shovel_l, xpos_pass_td_l, ypos_pass_td_l)
xps_td_r, yps_td_r = return_path(xpos_pass_shovel_r, ypos_pass_shovel_r, xpos_pass_td_r, ypos_pass_td_r)


#%%
def rotation(fx, fy, gx, gy):
    for i in range(0, len(fx)):
        if fx[i] != "empty":
            new_x = []
            for x in fx[i]:
                x_r = 120 - x
                new_x.append(x_r)
            gx.append(new_x)
            new_y = []
            for y in fy[i]:
                y_r = 53.3 - y
                new_y.append(y_r)
            gy.append(new_y)
    return gx, gy

xpf_c, ypf_c = rotation(xpf_c_l, ypf_c_l, xpf_c_r, ypf_c_r)
xpf_int, ypf_int = rotation(xpf_int_l, ypf_int_l, xpf_int_r, ypf_int_r)
xpf_inc, ypf_inc = rotation(xpf_inc_l, ypf_inc_l, xpf_inc_r, ypf_inc_r)
xpf_td, ypf_td = rotation(xpf_td_l, ypf_td_l, xpf_td_r, ypf_td_r)

xps_c, yps_c = rotation(xps_c_l, yps_c_l, xps_c_r, yps_c_r)
xps_int, yps_int = rotation(xps_int_l, yps_int_l, xps_int_r, yps_int_r)
xps_inc, yps_inc = rotation(xps_inc_l, yps_inc_l, xps_inc_r, yps_inc_r)
xps_td, yps_td = rotation(xps_td_l, yps_td_l, xps_td_r, yps_td_r)

KeyboardInterrupt: ignored

**Machine learning**


In [65]:
#print(tracking2018['event'].unique())
#print(plays['specialTeamsResult'].unique())
print(plays['passResult'].unique())

[nan 'I' 'C' 'IN' 'S']


In [89]:
plays.head(0)

,gameId,playId,playDescription,quarter,down,yardsToGo,possessionTeam,specialTeamsPlayType,specialTeamsResult,kickerId,...,preSnapHomeScore,preSnapVisitorScore,passResult,kickLength,kickReturnYardage,playResult,absoluteYardlineNumber,game_clock_minute,game_clock_second,game_clock_milli


In [93]:
# inidividual dat
I_data = plays.loc[plays['passResult']=="I"]
C_data = plays.loc[plays['passResult']=="C"]
IN_data = plays.loc[plays['passResult']=="IN"]
S_data = plays.loc[plays['passResult']=="S"]

# stacked data
all_data = pd.concat([I_data, C_data, IN_data, S_data], axis=0)

data = all_data[['quarter','down','yardsToGo','absoluteYardlineNumber','game_clock_minute']]

# checking
data.head()

,quarter,down,yardsToGo,absoluteYardlineNumber,game_clock_minute
1087,2,1,10,67,0
1251,3,4,10,74,11
1709,2,4,7,98,5
3356,2,4,9,84,6
5953,3,4,5,80,0


In [194]:
from tensorflow.keras.utils import to_categorical
import numpy as np
import cv2
from imutils import paths
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
import tensorflow as tf
from tensorflow.keras import layers
from tensorflow.keras.models import Sequential
from sklearn.metrics import classification_report
import torch
import torchvision
import torchvision.transforms as transforms
import matplotlib.pyplot as plt
     

features = np.array(data)

# pass outcome
labels = np.array([0]*len(I_data)+[1]*len(C_data)+[2]*len(IN_data)+[3]*len(S_data))

print("shape of features:", features.shape)
print("shape of labels:", labels.shape)
print("length of features:", len(features))
print("length of labels:", len(labels))

from tensorflow.keras.utils import to_categorical

shape of features: (45, 5)
shape of labels: (45,)
length of features: 45
length of labels: 45


In [195]:
#x_train, x_test, y_train, y_test = train_test_split(x, y, test_size = 0.3, random_state=1)
import numpy as np
from sklearn.model_selection import train_test_split
# Separate the test data
x_train, x_test, y_train, y_test = train_test_split(features, labels, test_size=0.3, random_state=42)

norm_y_test = y_test

y_train = to_categorical(y_train)
y_test = to_categorical(y_test)

# Split the remaining data to train and validation
#x_train, x_val, y_train, y_val = train_test_split(x, y, test_size=0.15, random_state=1)

In [222]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.optimizers import SGD
from tensorflow.keras.layers import Conv2D
from tensorflow.keras.layers import MaxPooling2D
from tensorflow.keras.layers import Flatten

# create model
# model = Sequenial()
# model.add(Dense(8, activation='relu', input_shape=(features.shape[1],)))
# model.add(Dense(8, activation='relu'))
# model.add(Dense(1, activation='sigmoid'))
# layers.Flatten(),
# layers.Dense(128, activation='relu'),
# layers.Dense(2)
# model.compile(loss='binary_crossentropy',
# optimizer='sgd',
# metrics=['accuracy'])
#history = model.fit(x_train, y_train,epochs=8, batch_size=1, verbose=1)

# model = Sequential([
#   #layers.experimental.preprocessing.Rescaling(1./255, input_shape=(50, 50, 3)),
#   layers.Conv2D(16, 1, padding='same', activation='relu'),
#   layers.MaxPooling2D(),
#   layers.Conv2D(32, 1, padding='same', activation='relu'),
#   layers.MaxPooling2D(),
#   layers.Conv2D(64, 1, padding='same', activation='relu'),
#   layers.MaxPooling2D(),
#   layers.Conv2D(128, 1, padding='same', activation='relu'),
#   layers.MaxPooling2D(),
#   layers.Flatten(),
#   layers.Dense(128, activation='relu'),
#   layers.Dense(2)
# ])

#model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.00001),loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),metrics=['accuracy']) # 'binary_crossentropy' tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)

#history = model.fit(x_train, y_train, epochs=10)

model = Sequential()
model.add(Dense(128, activation='relu', input_dim=features.shape[1]))
model.add(Dense(4, activation='softmax'))
model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.001), loss='binary_crossentropy', metrics=['accuracy'])
history = model.fit(x_train, y_train, epochs=100, batch_size=5, validation_split=0.2)

Epoch 1/100
5/5 [==============================] - 1s 52ms/step - loss: 7.0603 - accuracy: 0.0417 - val_loss: 4.5980 - val_accuracy: 0.1429
Epoch 2/100
5/5 [==============================] - 0s 12ms/step - loss: 4.8104 - accuracy: 0.0417 - val_loss: 2.8612 - val_accuracy: 0.1429
Epoch 3/100
5/5 [==============================] - 0s 12ms/step - loss: 2.9733 - accuracy: 0.0417 - val_loss: 1.6550 - val_accuracy: 0.1429
Epoch 4/100
5/5 [==============================] - 0s 14ms/step - loss: 1.6850 - accuracy: 0.0417 - val_loss: 0.8561 - val_accuracy: 0.2857
Epoch 5/100
5/5 [==============================] - 0s 16ms/step - loss: 0.7445 - accuracy: 0.1667 - val_loss: 0.4738 - val_accuracy: 0.4286
Epoch 6/100
5/5 [==============================] - 0s 15ms/step - loss: 0.5101 - accuracy: 0.5833 - val_loss: 0.5241 - val_accuracy: 0.2857
Epoch 7/100
5/5 [==============================] - 0s 15ms/step - loss: 0.5186 - accuracy: 0.5000 - val_loss: 0.5626 - val_accuracy: 0.4286
Epoch 8/100
5/5 [===

In [223]:
loss, accuracy = model.evaluate(x_test, y_test)

1/1 [==============================] - 0s 29ms/step - loss: 0.4292 - accuracy: 0.7143


In [226]:
from sklearn.metrics import f1_score, precision_score, recall_score, confusion_matrix
y_pred1 = model.predict(x_test)
y_pred = np.argmax(y_pred1, axis=1)

print("Predictions:")
print(y_pred)
print("Actual:")
print(norm_y_test)
print("")

1/1 [==============================] - 0s 25ms/step
Predictions:
[1 1 1 1 1 1 0 0 1 1 0 1 1 1]
Actual:
[1 1 1 3 1 2 0 0 0 0 0 1 1 1]


In [ ]:
!pip install plotly>=4.0.0
!wget https://github.com/plotly/orca/releases/download/v1.2.1/orca-1.2.1-x86_64.AppImage -O /usr/local/bin/orca
!chmod +x /usr/local/bin/orca
!apt-get install xvfb libgtk2.0-0 libgconf-2-4

In [246]:
import plotly.graph_objects as go
import plotly.express as px

fig = go.Figure()

data = history.history['accuracy']

fig.add_trace(go.Scatter(
    x=np.linspace(1,100,100),
    y=data,
    name="Accuracy",
    line=dict(color='black'),
    mode='lines',   
    #showlegend = True
))

fig.update_layout(
    width=600, 
    height=400,
    #xaxis_range=[],
    #yaxis_range=[],
    plot_bgcolor='white',
    xaxis_title="Epochs",
    yaxis_title="Accuracy",
    font=dict(
    family="Didone",
    size=18,  # Set the font size here
    color="Black",
    )
)

fig.update_xaxes(
    mirror=True,
    ticks='outside',
    showline=True,
    linecolor='black',
    gridcolor='lightgrey'
)
fig.update_yaxes(
    mirror=True,
    ticks='outside',
    showline=True,
    linecolor='black',
    gridcolor='lightgrey',
)

fig.update_layout(legend=dict(yanchor="bottom", y=0.7, xanchor="left", x=1.05))

fig.show()

fig.write_image("/content/nn_acc.pdf")